<a href="https://colab.research.google.com/github/iamZamil/Market-Anomaly-Detection/blob/main/Market_Anomaly_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Install necessary libraries
!pip install numpy pandas scikit-learn matplotlib

# Import libraries
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt


In [24]:
# Upload the dataset
from google.colab import files
uploaded = files.upload()

# Load the dataset into a DataFrame
import pandas as pd
import numpy as np

filename = list(uploaded.keys())[0]
data = pd.read_csv(filename)
print("Dataset Shape:", data.shape)
print(data.head())

# Check for missing values
print("Missing Values:\n", data.isnull().sum())


Saving dataset.csv to dataset (1).csv
Dataset Shape: (1153, 60)
  EURUSDV3M Index Unnamed: 1      Unnamed: 2 Fixed Income, MM & Interbank  \
0             NaN        NaN  Show/Hide data                            S   
1             676        NaN            Name                          NaN   
2             NaN        NaN  Index Currency                          NaN   
3             NaN        NaN          Ticker              XAU BGNL Curncy   
4             NaN        NaN             NaN                       284.25   

       Unnamed: 4  Unnamed: 5 Unnamed: 6  Unnamed: 7  Unnamed: 8  Unnamed: 9  \
0               H           H          H           H           H           H   
1             NaN         NaN        NaN         NaN         NaN         NaN   
2             NaN         NaN        NaN         NaN         NaN         NaN   
3  ECSURPUS Index  BDIY Index  CRY Index  DXY Curncy  JPY Curncy  GBP Curncy   
4             NaN         NaN        NaN         NaN         NaN         

In [8]:
print(data.shape)  # Check the shape of the data
print(data.head())  # Check the first few rows of the data


(1153, 60)
  EURUSDV3M Index Unnamed: 1      Unnamed: 2 Fixed Income, MM & Interbank  \
0             NaN        NaN  Show/Hide data                            S   
1             676        NaN            Name                          NaN   
2             NaN        NaN  Index Currency                          NaN   
3             NaN        NaN          Ticker              XAU BGNL Curncy   
4             NaN        NaN             NaN                       284.25   

       Unnamed: 4  Unnamed: 5 Unnamed: 6  Unnamed: 7  Unnamed: 8  Unnamed: 9  \
0               H           H          H           H           H           H   
1             NaN         NaN        NaN         NaN         NaN         NaN   
2             NaN         NaN        NaN         NaN         NaN         NaN   
3  ECSURPUS Index  BDIY Index  CRY Index  DXY Curncy  JPY Curncy  GBP Curncy   
4             NaN         NaN        NaN         NaN         NaN         NaN   

   ...    Treasury        Gold       Brent   

In [10]:
# Remove all "Unnamed:" columns
data_clean = data.loc[:, ~data.columns.str.contains('^Unnamed')]

# Optionally, reset column names with generic names if needed
data_clean.columns = ['Column' + str(i+1) for i in range(data_clean.shape[1])]

# Check the first few rows after cleaning
print(data_clean.head())


  Column1          Column2     Column3     Column4     Column5     Column6  \
0     NaN                S           H           H           H           H   
1     676              NaN         NaN         NaN         NaN         NaN   
2     NaN              NaN         NaN         NaN         NaN         NaN   
3     NaN  XAU BGNL Curncy  RX1 Comdty  TY1 Comdty  GC1 Comdty  CO1 Comdty   
4     NaN           284.25         NaN         NaN         NaN         NaN   

     Column7    Column8    Column9    Column10   Column11    Column12  \
0          H          H          H           H          H           H   
1        NaN        NaN        NaN         NaN        NaN         NaN   
2        NaN        NaN        NaN         NaN        NaN         NaN   
3  ES1 Index  VG1 Index  NQ1 Index  LLL1 Index  TP1 Index  DU1 Comdty   
4        NaN        NaN        NaN         NaN        NaN         NaN   

     Column13  
0           H  
1         NaN  
2         NaN  
3  TU2 Comdty  
4         Na

In [11]:
print(data_clean.columns)


Index(['Column1', 'Column2', 'Column3', 'Column4', 'Column5', 'Column6',
       'Column7', 'Column8', 'Column9', 'Column10', 'Column11', 'Column12',
       'Column13'],
      dtype='object')


In [13]:
# Option 1: Fill missing values for numeric columns with their mean
numeric_columns = data_clean.select_dtypes(include=['number']).columns
data_clean[numeric_columns] = data_clean[numeric_columns].fillna(data_clean[numeric_columns].mean())

# Option 2: Fill missing values for non-numeric columns with a constant (e.g., 'Unknown')
non_numeric_columns = data_clean.select_dtypes(exclude=['number']).columns
data_clean[non_numeric_columns] = data_clean[non_numeric_columns].fillna('Unknown')

# Check the cleaned data
print(data_clean.head())


   Column1          Column2     Column3     Column4     Column5     Column6  \
0  Unknown                S           H           H           H           H   
1      676          Unknown     Unknown     Unknown     Unknown     Unknown   
2  Unknown          Unknown     Unknown     Unknown     Unknown     Unknown   
3  Unknown  XAU BGNL Curncy  RX1 Comdty  TY1 Comdty  GC1 Comdty  CO1 Comdty   
4  Unknown           284.25     Unknown     Unknown     Unknown     Unknown   

     Column7    Column8    Column9    Column10   Column11    Column12  \
0          H          H          H           H          H           H   
1    Unknown    Unknown    Unknown     Unknown    Unknown     Unknown   
2    Unknown    Unknown    Unknown     Unknown    Unknown     Unknown   
3  ES1 Index  VG1 Index  NQ1 Index  LLL1 Index  TP1 Index  DU1 Comdty   
4    Unknown    Unknown    Unknown     Unknown    Unknown     Unknown   

     Column13  
0           H  
1     Unknown  
2     Unknown  
3  TU2 Comdty  
4     

<ipython-input-13-1e05e13fe027>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean[non_numeric_columns] = data_clean[non_numeric_columns].fillna('Unknown')


In [14]:
# Check the shape and first few rows
print(data_clean.shape)
print(data_clean.head())


(1153, 13)
   Column1          Column2     Column3     Column4     Column5     Column6  \
0  Unknown                S           H           H           H           H   
1      676          Unknown     Unknown     Unknown     Unknown     Unknown   
2  Unknown          Unknown     Unknown     Unknown     Unknown     Unknown   
3  Unknown  XAU BGNL Curncy  RX1 Comdty  TY1 Comdty  GC1 Comdty  CO1 Comdty   
4  Unknown           284.25     Unknown     Unknown     Unknown     Unknown   

     Column7    Column8    Column9    Column10   Column11    Column12  \
0          H          H          H           H          H           H   
1    Unknown    Unknown    Unknown     Unknown    Unknown     Unknown   
2    Unknown    Unknown    Unknown     Unknown    Unknown     Unknown   
3  ES1 Index  VG1 Index  NQ1 Index  LLL1 Index  TP1 Index  DU1 Comdty   
4    Unknown    Unknown    Unknown     Unknown    Unknown     Unknown   

     Column13  
0           H  
1     Unknown  
2     Unknown  
3  TU2 Comd

In [17]:
# Check the data types of columns in the cleaned dataset
print(data_clean.dtypes)

# Select only numeric columns from the cleaned data
numeric_data = data_clean.select_dtypes(include=['number'])

# Check if numeric_data is empty
if numeric_data.empty:
    print("No numeric columns available for scaling.")
else:
    # Apply scaling to numeric data
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(numeric_data)

    # Convert the scaled data back into a DataFrame
    scaled_data_df = pd.DataFrame(scaled_data, columns=numeric_data.columns)

    # Display the scaled data
    print(scaled_data_df.head())


Column1     object
Column2     object
Column3     object
Column4     object
Column5     object
Column6     object
Column7     object
Column8     object
Column9     object
Column10    object
Column11    object
Column12    object
Column13    object
dtype: object
No numeric columns available for scaling.


In [19]:
print("Missing Values in Each Column:\n", data.isnull().sum())


Missing Values in Each Column:
 EURUSDV3M Index                 0
Unnamed: 1                      0
Unnamed: 2                      0
Fixed Income, MM & Interbank    0
Unnamed: 4                      0
Unnamed: 5                      0
Unnamed: 6                      0
Unnamed: 7                      0
Unnamed: 8                      0
Unnamed: 9                      0
Unnamed: 10                     0
Unnamed: 11                     0
Unnamed: 12                     0
Unnamed: 13                     0
Unnamed: 14                     0
Unnamed: 15                     0
Unnamed: 16                     0
Unnamed: 17                     0
Unnamed: 18                     0
Unnamed: 19                     0
Unnamed: 20                     0
Unnamed: 21                     0
Unnamed: 22                     0
Unnamed: 23                     0
Unnamed: 24                     0
Unnamed: 25                     0
Unnamed: 26                     0
Unnamed: 27                     0
Unnamed: 28     

In [21]:
data = data.fillna(data.mean())
print(data)

Empty DataFrame
Columns: [EURUSDV3M Index, Unnamed: 1, Unnamed: 2, Fixed Income, MM & Interbank, Unnamed: 4, Unnamed: 5, Unnamed: 6, Unnamed: 7, Unnamed: 8, Unnamed: 9, Unnamed: 10, Unnamed: 11, Unnamed: 12, Unnamed: 13, Unnamed: 14, Unnamed: 15, Unnamed: 16, Unnamed: 17, Unnamed: 18, Unnamed: 19, Unnamed: 20, Unnamed: 21, Unnamed: 22, Unnamed: 23, Unnamed: 24, Unnamed: 25, Unnamed: 26, Unnamed: 27, Unnamed: 28, Unnamed: 29, Unnamed: 30, Unnamed: 31, Unnamed: 32, Unnamed: 33, Unnamed: 34, Unnamed: 35, Unnamed: 36, Unnamed: 37, Unnamed: 38, Unnamed: 39, Unnamed: 40, Unnamed: 41, Unnamed: 42, Unnamed: 43, Unnamed: 44, Unnamed: 45, Unnamed: 46, Unnamed: 47, Unnamed: 48, Bund, Treasury, Gold, Brent, S&P, Eurostoxx, Nasdaq, EM, Japan, 2Y Ger, 2Y US]
Index: []

[0 rows x 60 columns]


In [23]:
print("Shape of Data:", data.shape)


Shape of Data: (0, 60)


In [26]:
# Replace 'Unknown' with NaN
data_clean = data_clean.replace('Unknown', np.nan)

# Drop rows with NaN values
data_clean = data_clean.dropna()

# Check the shape after dropping rows with NaN values
print("Shape after handling 'Unknown':", data_clean.shape)


Shape after handling 'Unknown': (0, 13)


In [31]:
# Try converting all columns to numeric, forcing errors to NaN (this will handle any non-numeric values)
data_clean = data.apply(pd.to_numeric, errors='coerce')

# Check the data types after conversion
print("Data Types After Conversion:\n", data_clean.dtypes)

# Now check for numeric columns again
numeric_columns = data_clean.select_dtypes(include=['float64', 'int64']).columns

# Display the columns to use for scaling
print("Numeric Columns After Conversion:\n", numeric_columns)

# Clean the data by replacing 'Unknown' (if present) with 0 or other placeholders
data_clean_filled = data_clean.fillna(0)

# Verify the cleaned data
print("Cleaned Data Sample:\n", data_clean_filled.head())

# Proceed with scaling if there are numeric columns left
if len(numeric_columns) > 0:
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(data_clean_filled[numeric_columns])

    # Convert the scaled data back to a DataFrame for readability
    scaled_data_df = pd.DataFrame(scaled_data, columns=numeric_columns)
    print("Scaled Data Sample:\n", scaled_data_df.head())
else:
    print("No numeric columns available for scaling.")


Data Types After Conversion:
 EURUSDV3M Index                 float64
Unnamed: 1                      float64
Unnamed: 2                      float64
Fixed Income, MM & Interbank    float64
Unnamed: 4                      float64
Unnamed: 5                      float64
Unnamed: 6                      float64
Unnamed: 7                      float64
Unnamed: 8                      float64
Unnamed: 9                      float64
Unnamed: 10                     float64
Unnamed: 11                     float64
Unnamed: 12                     float64
Unnamed: 13                     float64
Unnamed: 14                     float64
Unnamed: 15                     float64
Unnamed: 16                     float64
Unnamed: 17                     float64
Unnamed: 18                     float64
Unnamed: 19                     float64
Unnamed: 20                     float64
Unnamed: 21                     float64
Unnamed: 22                     float64
Unnamed: 23                     float64
Unnamed: 2

In [32]:
# Select only numeric columns
data_clean_numeric = data_clean.select_dtypes(include=['float64', 'int64'])

# Check the shape of the numeric data
print("Shape of numeric data:", data_clean_numeric.shape)

# Proceed with scaling
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data_clean_numeric)

# Convert the scaled data back to a DataFrame for readability
scaled_data_df = pd.DataFrame(scaled_data, columns=data_clean_numeric.columns)
print("Scaled Data Sample:\n", scaled_data_df.head())


Shape of numeric data: (1153, 60)
Scaled Data Sample:
    EURUSDV3M Index  Unnamed: 1  Unnamed: 2  Fixed Income, MM & Interbank  \
0              NaN         NaN         NaN                           NaN   
1              0.0         NaN         NaN                           NaN   
2              NaN         NaN         NaN                           NaN   
3              NaN         NaN         NaN                           NaN   
4              NaN         NaN         NaN                     -1.352549   

   Unnamed: 4  Unnamed: 5  Unnamed: 6  Unnamed: 7  Unnamed: 8  Unnamed: 9  \
0         NaN         NaN         NaN         NaN         NaN         NaN   
1         NaN         NaN         NaN         NaN         NaN         NaN   
2         NaN         NaN         NaN         NaN         NaN         NaN   
3         NaN         NaN         NaN         NaN         NaN         NaN   
4         NaN         NaN         NaN         NaN         NaN         NaN   

   ...  Treasury  Gold  B

/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1101: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1106: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1126: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


In [33]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_data = scaler.fit_transform(data_clean)

# Convert the scaled data back to a DataFrame for readability
scaled_data_df = pd.DataFrame(scaled_data, columns=data_clean.columns)
print("Scaled Data Sample:\n", scaled_data_df.head())


Scaled Data Sample:
    EURUSDV3M Index  Unnamed: 1  Unnamed: 2  Fixed Income, MM & Interbank  \
0              NaN         NaN         NaN                           NaN   
1              0.0         NaN         NaN                           NaN   
2              NaN         NaN         NaN                           NaN   
3              NaN         NaN         NaN                           NaN   
4              NaN         NaN         NaN                     -1.352549   

   Unnamed: 4  Unnamed: 5  Unnamed: 6  Unnamed: 7  Unnamed: 8  Unnamed: 9  \
0         NaN         NaN         NaN         NaN         NaN         NaN   
1         NaN         NaN         NaN         NaN         NaN         NaN   
2         NaN         NaN         NaN         NaN         NaN         NaN   
3         NaN         NaN         NaN         NaN         NaN         NaN   
4         NaN         NaN         NaN         NaN         NaN         NaN   

   ...  Treasury  Gold  Brent  S&P  Eurostoxx  Nasdaq  EM  

/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1101: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1106: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1126: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


In [36]:
from sklearn.ensemble import IsolationForest
import pandas as pd

# Assuming `scaled_data_df` is the DataFrame containing the scaled data (from previous steps)
# Train the Isolation Forest model
isolation_forest = IsolationForest(random_state=42, contamination=0.05)
labels = isolation_forest.fit_predict(scaled_data_df)

# Add anomaly labels to the DataFrame
scaled_data_df['IsolationForest_Label'] = labels

# Print the number of anomalies detected by Isolation Forest (label = -1 means anomaly)
print("Isolation Forest Anomalies:", (labels == -1).sum())

# Optionally, display the first few rows of the resulting DataFrame
print(scaled_data_df.head())


Isolation Forest Anomalies: 58
   EURUSDV3M Index  Unnamed: 1  Unnamed: 2  Fixed Income, MM & Interbank  \
0              NaN         NaN         NaN                           NaN   
1              0.0         NaN         NaN                           NaN   
2              NaN         NaN         NaN                           NaN   
3              NaN         NaN         NaN                           NaN   
4              NaN         NaN         NaN                     -1.352549   

   Unnamed: 4  Unnamed: 5  Unnamed: 6  Unnamed: 7  Unnamed: 8  Unnamed: 9  \
0         NaN         NaN         NaN         NaN         NaN         NaN   
1         NaN         NaN         NaN         NaN         NaN         NaN   
2         NaN         NaN         NaN         NaN         NaN         NaN   
3         NaN         NaN         NaN         NaN         NaN         NaN   
4         NaN         NaN         NaN         NaN         NaN         NaN   

   ...  Gold  Brent  S&P  Eurostoxx  Nasdaq  EM  

In [46]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import LocalOutlierFactor
from google.colab import files
uploaded = files.upload()

# Load the dataset into a DataFrame
import pandas as pd
import numpy as np

filename = list(uploaded.keys())[0]
data = pd.read_csv(filename)
# Load your dataset (replace 'your_data.csv' with your actual file path)
df = pd.read_csv(filename)

# Get the column names before scaling
column_names = df.columns

# Apply scaling to your data (if you haven't already)

# Convert scaled data back to DataFrame with the original column names
scaled_data = pd.DataFrame(scaled_data, columns=column_names)

# Handle missing features (drop columns that are all NaN)
scaled_data = scaled_data.dropna(axis=1, how='all')  # Drop columns with all NaN values

# Impute missing values with the mean of each column
imputer = SimpleImputer(strategy='mean')
scaled_data_imputed = imputer.fit_transform(scaled_data)

# Now, train the LOF model
lof = LocalOutlierFactor(n_neighbors=20, contamination=0.05)
lof_labels = lof.fit_predict(scaled_data_imputed)

# Convert the imputed data back to a DataFrame
scaled_data_imputed_df = pd.DataFrame(scaled_data_imputed, columns=scaled_data.columns)

# Add anomaly labels to the DataFrame
scaled_data_imputed_df['LOF_Label'] = lof_labels
print("LOF Anomalies:", (lof_labels == -1).sum())


Saving dataset.csv to dataset (3).csv
LOF Anomalies: 58


In [50]:
import pandas as pd
from sklearn.covariance import EllipticEnvelope
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# Assuming you already have a DataFrame `df`
# If you need to load a CSV file, you would use:
# df = pd.read_csv('path_to_your_file.csv')

# Check the column types and see if there are numeric columns
print("Column types in the dataset:")
print(df.dtypes)

# Filter out non-numeric columns (only use numeric columns for scaling)
numeric_df = df.select_dtypes(include=['number'])

# If numeric_df is empty, print a message and exit
if numeric_df.empty:
    print("No numeric columns found in the dataset.")
else:
    # Apply scaling to the numeric columns only
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(numeric_df)

    # Handle missing features (drop columns that are all NaN)
    scaled_data = pd.DataFrame(scaled_data, columns=numeric_df.columns)
    scaled_data = scaled_data.dropna(axis=1, how='all')  # Drop columns with all NaN values

    # Impute missing values with the mean of each column
    imputer = SimpleImputer(strategy='mean')
    scaled_data_imputed = imputer.fit_transform(scaled_data)

    # Now, train the robust covariance model (Elliptic Envelope)
    cov = EllipticEnvelope(random_state=42, contamination=0.05)
    cov_labels = cov.fit_predict(scaled_data_imputed)

    # Convert the imputed data back to a DataFrame
    scaled_data_imputed_df = pd.DataFrame(scaled_data_imputed, columns=scaled_data.columns)

    # Add anomaly labels to the DataFrame
    scaled_data_imputed_df['RobustCovariance_Label'] = cov_labels
    print("Robust Covariance Anomalies:", (cov_labels == -1).sum())


Column types in the dataset:
EURUSDV3M Index                 object
Unnamed: 1                      object
Unnamed: 2                      object
Fixed Income, MM & Interbank    object
Unnamed: 4                      object
Unnamed: 5                      object
Unnamed: 6                      object
Unnamed: 7                      object
Unnamed: 8                      object
Unnamed: 9                      object
Unnamed: 10                     object
Unnamed: 11                     object
Unnamed: 12                     object
Unnamed: 13                     object
Unnamed: 14                     object
Unnamed: 15                     object
Unnamed: 16                     object
Unnamed: 17                     object
Unnamed: 18                     object
Unnamed: 19                     object
Unnamed: 20                     object
Unnamed: 21                     object
Unnamed: 22                     object
Unnamed: 23                     object
Unnamed: 24                     obj

In [51]:
import pandas as pd
from sklearn.covariance import EllipticEnvelope
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# Assuming you already have a DataFrame `df`
# If you need to load a CSV file, you would use:
# df = pd.read_csv('path_to_your_file.csv')

# Check the column types and see if there are numeric columns
print("Column types in the dataset:")
print(df.dtypes)

# Filter out non-numeric columns (only use numeric columns for scaling)
numeric_df = df.select_dtypes(include=['number'])

# If numeric_df is empty, print a message and exit
if numeric_df.empty:
    print("No numeric columns found in the dataset.")
else:
    # Apply scaling to the numeric columns only
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(numeric_df)

    # Handle missing features (drop columns that are all NaN)
    scaled_data = pd.DataFrame(scaled_data, columns=numeric_df.columns)
    scaled_data = scaled_data.dropna(axis=1, how='all')  # Drop columns with all NaN values

    # Impute missing values with the mean of each column
    imputer = SimpleImputer(strategy='mean')
    scaled_data_imputed = imputer.fit_transform(scaled_data)

    # Now, train the robust covariance model (Elliptic Envelope)
    cov = EllipticEnvelope(random_state=42, contamination=0.05)
    cov_labels = cov.fit_predict(scaled_data_imputed)

    # Convert the imputed data back to a DataFrame
    scaled_data_imputed_df = pd.DataFrame(scaled_data_imputed, columns=scaled_data.columns)

    # Add anomaly labels to the DataFrame
    scaled_data_imputed_df['RobustCovariance_Label'] = cov_labels
    print("Robust Covariance Anomalies:", (cov_labels == -1).sum())


Column types in the dataset:
EURUSDV3M Index                 object
Unnamed: 1                      object
Unnamed: 2                      object
Fixed Income, MM & Interbank    object
Unnamed: 4                      object
Unnamed: 5                      object
Unnamed: 6                      object
Unnamed: 7                      object
Unnamed: 8                      object
Unnamed: 9                      object
Unnamed: 10                     object
Unnamed: 11                     object
Unnamed: 12                     object
Unnamed: 13                     object
Unnamed: 14                     object
Unnamed: 15                     object
Unnamed: 16                     object
Unnamed: 17                     object
Unnamed: 18                     object
Unnamed: 19                     object
Unnamed: 20                     object
Unnamed: 21                     object
Unnamed: 22                     object
Unnamed: 23                     object
Unnamed: 24                     obj

In [56]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.svm import OneClassSVM

# Assuming you already have a DataFrame `df`
# If you need to load a CSV file, you would use:
# df = pd.read_csv('path_to_your_file.csv')

# Check the columns and their types
print("Columns in df:", df.columns)
print("Data types in df:", df.dtypes)

# Select only numeric columns
numeric_df = df.select_dtypes(include=['number'])

# Check if numeric columns exist
if  numeric_df.empty:
    print("No numeric columns found in the DataFrame.")
else:
    # Apply scaling to the numeric columns only
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(numeric_df)

    # Handle missing values (e.g., impute missing values with mean)
    imputer = SimpleImputer(strategy='mean')
    scaled_data_imputed = imputer.fit_transform(scaled_data)

    # Ensure no NaN values remain after imputation
    if pd.isna(scaled_data_imputed).any():
        print("Data still contains NaN values!")
    else:
        # Train the One-Class SVM model
        oc_svm = OneClassSVM(kernel='rbf', gamma=0.1, nu=0.05)
        oc_svm_labels = oc_svm.fit_predict(scaled_data_imputed)

        # Add anomaly labels to the DataFrame
        scaled_data_df = pd.DataFrame(scaled_data_imputed, columns=numeric_df.columns)
        scaled_data_df['OneClassSVM_Label'] = oc_svm_labels

        # Print the number of anomalies detected
        print("One-Class SVM Anomalies:", (oc_svm_labels == -1).sum())


Columns in df: Index(['EURUSDV3M Index', 'Unnamed: 1', 'Unnamed: 2',
       'Fixed Income, MM & Interbank', 'Unnamed: 4', 'Unnamed: 5',
       'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10',
       'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14',
       'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18',
       'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22',
       'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26',
       'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30',
       'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34',
       'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38',
       'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41', 'Unnamed: 42',
       'Unnamed: 43', 'Unnamed: 44', 'Unnamed: 45', 'Unnamed: 46',
       'Unnamed: 47', 'Unnamed: 48', 'Bund', 'Treasury', 'Gold', 'Brent',
       'S&P', 'Eurostoxx', 'Nasdaq', 'EM', 'Japan', '2Y Ger', '2Y US'],
      dtype='object')
Data types in d

In [57]:
# For time-series datasets
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Prepare data for LSTM (assuming a time-series format)
timesteps = 10
X = []
for i in range(len(scaled_data) - timesteps):
    X.append(scaled_data.iloc[i:i+timesteps].values)

X = np.array(X)

# Define the LSTM model
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(timesteps, X.shape[2])),
    Dropout(0.2),
    LSTM(32, return_sequences=False),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X, X[:, -1], epochs=10, batch_size=32, validation_split=0.2)

# Predict anomalies
predictions = model.predict(X)
anomaly_scores = np.mean(np.abs(X[:, -1] - predictions), axis=1)
threshold = np.percentile(anomaly_scores, 95)
lstm_anomalies = anomaly_scores > threshold
print("LSTM Anomalies:", lstm_anomalies.sum())


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 6s 58ms/step - loss: nan - val_loss: nan
Epoch 2/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: nan - val_loss: nan
Epoch 3/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: nan - val_loss: nan
Epoch 4/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: nan - val_loss: nan
Epoch 5/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: nan - val_loss: nan
Epoch 6/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: nan - val_loss: nan
Epoch 7/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: nan - val_loss: nan
Epoch 8/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: nan - val_loss: nan
Epoch 9/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: nan - val_loss: nan
Epoch 10/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: nan - val_loss: nan
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
LSTM Anomalies: 0


In [59]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest

# Assuming you already have a DataFrame `df`

# Check the columns and their types
print("Columns in df:", df.columns)
print("Data types in df:", df.dtypes)

# Select only numeric columns
numeric_df = df.select_dtypes(include=['number'])

# Check if numeric columns exist
if numeric_df.empty:
    print("No numeric columns found in the DataFrame.")
else:
    # Apply scaling to the numeric columns only
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(numeric_df)

    # Handle missing values (e.g., impute missing values with mean)
    imputer = SimpleImputer(strategy='mean')
    scaled_data_imputed = imputer.fit_transform(scaled_data)

    # Ensure no NaN values remain after imputation
    if pd.isna(scaled_data_imputed).any():
        print("Data still contains NaN values!")
    else:
        # Train the One-Class SVM model
        oc_svm = OneClassSVM(kernel='rbf', gamma=0.1, nu=0.05)
        oc_svm_labels = oc_svm.fit_predict(scaled_data_imputed)

        # Add anomaly labels to the DataFrame
        scaled_data_df = pd.DataFrame(scaled_data_imputed, columns=numeric_df.columns)
        scaled_data_df['OneClassSVM_Label'] = oc_svm_labels

        # Train the Isolation Forest model
        isolation_forest = IsolationForest(contamination=0.05, random_state=42)
        isolation_forest_labels = isolation_forest.fit_predict(scaled_data_imputed)

        # Add the Isolation Forest anomaly labels to the DataFrame
        scaled_data_df['IsolationForest_Label'] = isolation_forest_labels

        # Print the number of anomalies detected by both models
        print("One-Class SVM Anomalies:", (oc_svm_labels == -1).sum())
        print("Isolation Forest Anomalies:", (isolation_forest_labels == -1).sum())

        # Optional: Display the first few rows of the DataFrame with the labels
        print(scaled_data_df.head())


Columns in df: Index(['EURUSDV3M Index', 'Unnamed: 1', 'Unnamed: 2',
       'Fixed Income, MM & Interbank', 'Unnamed: 4', 'Unnamed: 5',
       'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10',
       'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14',
       'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18',
       'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22',
       'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26',
       'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30',
       'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34',
       'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38',
       'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41', 'Unnamed: 42',
       'Unnamed: 43', 'Unnamed: 44', 'Unnamed: 45', 'Unnamed: 46',
       'Unnamed: 47', 'Unnamed: 48', 'Bund', 'Treasury', 'Gold', 'Brent',
       'S&P', 'Eurostoxx', 'Nasdaq', 'EM', 'Japan', '2Y Ger', '2Y US'],
      dtype='object')
Data types in d